In [9]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import numpy as np

In [10]:
data_version = "06-01-2021-v4"

In [11]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version}/test.csv")

In [12]:
df_train

,id,label,Field_3,Field_4,Field_10,Field_12,Field_13,Field_19,Field_20,Field_21,...,Field_7Field_8,Field_7Field_9,Field_8Field_9,F_startDateF_endDate,E_startDateE_endDate,C_startDateC_endDate,G_startDateG_endDate,A_startDateA_endDate,gender,Age
0,0,1,1.0,0,1.0,3,1,0.0,4258600.0,4.5,...,211.0,-5.0,-216.0,NaN,NaN,NaN,NaN,67.0,1,29.0
1,1,0,NaN,1,NaN,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,NaN
2,2,0,2.0,2,1.0,6,1,0.0,5000000.0,4.5,...,364.0,16.0,-348.0,0.0,24.0,NaN,NaN,47.0,0,22.0
3,3,0,NaN,1,NaN,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
4,4,1,NaN,1,NaN,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53025,53025,0,1.0,0,1.0,6,1,1.0,1490000.0,4.5,...,365.0,-8.0,-373.0,0.0,0.0,0.0,NaN,202.0,0,32.0
53026,53026,0,NaN,1,NaN,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.0,2,NaN
53027,53027,0,2.0,2,1.0,3,1,0.0,4015000.0,4.5,...,327.0,196.0,-131.0,NaN,NaN,NaN,NaN,NaN,0,32.0
53028,53028,0,NaN,1,NaN,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,NaN


In [13]:
y = df_train.label
train = df_train.drop(["label"], axis = 1)

In [14]:
def gini(y_true, y_score):
    return roc_auc_score(y_true, y_score)*2 - 1

def lgb_gini(y_pred, dataset_true):
    y_true = dataset_true.get_label()
    return 'gini', gini(y_true, y_pred), True

In [8]:
zzzz

In [9]:
for col in cols:
    vc = train[col].value_counts()
    if len(vc) <= 3:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')

In [17]:
lgbm_param = {'boosting_type': 'gbdt',\
              'colsample_bytree': 0.6602479798930369, \
              'is_unbalance': False, \
              'learning_rate': 0.001, \
              'max_depth': 15, \
              'metric': 'auc', \
              'min_child_samples': 25, \
              'num_leaves': 60, \
              'objective': 'binary', \
              'reg_alpha': 0.4693391197064131, \
              'reg_lambda': 0.16175478669541327, \
              'subsample_for_bin': 60000}

In [18]:
NUM_BOOST_ROUND= 10000

In [19]:
def kfold(train_fe,y_label,test_fe):
    seeds = [1212,123,2134,123345,123123]
    preds = 0    
    feature_important = None
    avg_train_gini = 0
    avg_val_gini = 0

    for s in seeds:
        skf = StratifiedKFold(n_splits=5, random_state = 6484, shuffle=True)        
        lgbm_param['random_state'] = s   
        seed_train_gini = 0
        seed_val_gini = 0
        for i, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(y_label)), y_label)):                
            X_train, X_val = train_fe.iloc[train_idx].drop(["id"], 1), train_fe.iloc[val_idx].drop(["id"], 1)                
            y_train, y_val = y_label[train_idx], y_label[val_idx]

            lgb_train = lgb.Dataset(X_train, y_train)
            lgb_eval  = lgb.Dataset(X_val, y_val)

            evals_result = {} 
            model = lgb.train(lgbm_param,
                        lgb_train,
                        num_boost_round=NUM_BOOST_ROUND,  
                        early_stopping_rounds=400,
                        feval=lgb_gini,
                        verbose_eval= 200,
                        evals_result=evals_result,
                        valid_sets=[lgb_train, lgb_eval])

            seed_train_gini += model.best_score["training"]["gini"] / skf.n_splits
            seed_val_gini += model.best_score["valid_1"]["gini"] / skf.n_splits

            avg_train_gini += model.best_score["training"]["gini"] / (len(seeds) * skf.n_splits)
            avg_val_gini += model.best_score["valid_1"]["gini"] / (len(seeds) * skf.n_splits)

            if feature_important is None:
                feature_important = model.feature_importance() / (len(seeds) * skf.n_splits)
            else:
                feature_important += model.feature_importance() / (len(seeds) * skf.n_splits)        

            pred = model.predict(test_fe.drop(["id"], 1))
            preds += pred / (skf.n_splits * len(seeds))

            print("Fold {}: {}/{}".format(i, model.best_score["training"]["gini"], model.best_score["valid_1"]["gini"]))
        print("Seed {}: {}/{}".format(s, seed_train_gini, seed_val_gini))

    print("-" * 30)
    print("Avg train gini: {}".format(avg_train_gini))
    print("Avg valid gini: {}".format(avg_val_gini))
    print("=" * 30)
    return preds

In [ ]:
train = df_train.drop(["id"], axis = 1)

In [22]:
preds = kfold(train, df_train.label, df_test)

Training until validation scores don't improve for 400 rounds
[200]	training's auc: 0.748802	training's gini: 0.497604	valid_1's auc: 0.723728	valid_1's gini: 0.447456
[400]	training's auc: 0.752337	training's gini: 0.504673	valid_1's auc: 0.724574	valid_1's gini: 0.449148
[600]	training's auc: 0.757996	training's gini: 0.515991	valid_1's auc: 0.728123	valid_1's gini: 0.456246


KeyboardInterrupt: 

In [14]:
"""
preds2 = preds
for i in range(len(preds)):
    if preds[i] >= 0.5:
        preds2[i] = 1
    else:
        preds2[i] = 0
"""
new_label = pd.DataFrame(preds, columns = ["label"])

In [15]:
new_data = pd.concat([df_test.id, new_label,df_test.iloc[:,1:]], axis = 1)

In [17]:
new_data

,id,label,Field_3,Field_4,Field_10,Field_12,Field_13,Field_18,Field_19,Field_20,...,Field_7Field_8,Field_7Field_9,Field_8Field_9,F_startDateF_endDate,E_startDateE_endDate,C_startDateC_endDate,G_startDateG_endDate,A_startDateA_endDate,gender,Age
0,53030,0.253576,2.0,t1,1.0,NaN,1,NaN,0.000,1490000.0,...,365.0,-28.0,-393.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,MALE,25.0
1,53031,0.455351,2.0,t1,1.0,NaN,1,NaN,0.000,1043000.0,...,365.0,-4.0,-369.0,-99999.0,-99999.0,-99999.0,-99999.0,0.0,MALE,28.0
2,53032,0.405164,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,...,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,missing_gender,-99999.0
3,53033,0.131885,2.0,t1,1.0,NaN,1,NaN,0.000,1490000.0,...,365.0,-25.0,-390.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,FEMALE,24.0
4,53034,0.109781,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,...,-99999.0,-99999.0,-99999.0,0.0,0.0,0.0,0.0,51.0,missing_gender,-99999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,0.218960,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,...,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,missing_gender,-99999.0
20377,73407,0.193544,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,...,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,missing_gender,-99999.0
20378,73408,0.099352,1.0,gh,1.0,NaN,1,NaN,4.136,6162640.0,...,730.0,-14.0,-744.0,-99999.0,0.0,-99999.0,-99999.0,0.0,FEMALE,38.0
20379,73409,0.227471,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,...,-99999.0,-99999.0,-99999.0,0.0,0.0,-99999.0,0.0,1.0,missing_gender,-99999.0


In [5]:
df_test

,id,Field_3,Field_4,Field_10,Field_12,Field_13,Field_18,Field_19,Field_20,Field_21,...,Field_7Field_8,Field_7Field_9,Field_8Field_9,F_startDateF_endDate,E_startDateE_endDate,C_startDateC_endDate,G_startDateG_endDate,A_startDateA_endDate,gender,Age
0,53030,2.0,t1,1.0,NaN,1,NaN,0.000,1490000.0,4.5,...,365.0,-28.0,-393.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,MALE,25.0
1,53031,2.0,t1,1.0,NaN,1,NaN,0.000,1043000.0,4.5,...,365.0,-4.0,-369.0,-99999.0,-99999.0,-99999.0,-99999.0,0.0,MALE,28.0
2,53032,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,-99999.0,...,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,missing_gender,-99999.0
3,53033,2.0,t1,1.0,NaN,1,NaN,0.000,1490000.0,4.5,...,365.0,-25.0,-390.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,FEMALE,24.0
4,53034,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,-99999.0,...,-99999.0,-99999.0,-99999.0,0.0,0.0,0.0,0.0,51.0,missing_gender,-99999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,-99999.0,...,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,missing_gender,-99999.0
20377,73407,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,-99999.0,...,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,missing_gender,-99999.0
20378,73408,1.0,gh,1.0,NaN,1,NaN,4.136,6162640.0,4.5,...,730.0,-14.0,-744.0,-99999.0,0.0,-99999.0,-99999.0,0.0,FEMALE,38.0
20379,73409,-99999.0,NaN,-99999.0,NaN,0,NaN,-99999.000,-99999.0,-99999.0,...,-99999.0,-99999.0,-99999.0,0.0,0.0,-99999.0,0.0,1.0,missing_gender,-99999.0


In [16]:
new_data.to_csv(f"../../data/kalapa/{data_version}/new_train.csv", index = False)